# Prevedere il crime rate

## Caricamento Librerie
Per prima cosa carichiamo le librerie per effettuare operazioni sui dati

*   NumPy per creare e operare su array a N dimensioni
*   pandas per caricare e manipolare dati tabulari
*   matplotlib per creare grafici

Importiamo le librerie usando i loro alias convenzionali

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

## Caricamento dei dati
I dataset presi sono satti scricati dal sito ufficile delle nazioni unite https://dataunodc.un.org/ , e contengono varie informazioni tra cui:
*  Kidnapping
*  Rape
*  Drug Trafficking
*  Sexual assault
*  Burglary and Theft (insieme)
*  Homicide
I dataset presentano tutti dati dal 2013 al 2022 delle varie nazioni del mondo

### Significato delle colonne
In tutti i dataset che prendiamo in considerazione abbiamo le varie colonne:
*  Nation
*

## Crimini per furto

In [61]:
import os.path
file = "data_cts_corruption_and_economic_crime.csv"
if not os.path.exists(file):
    print("Missing dataset")
else:
  theftCrime = pd.read_csv(file, index_col=False, encoding='latin1')

In [62]:
theftCrime.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22754 entries, 0 to 22753
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Iso3_code            22754 non-null  object 
 1   Country              22754 non-null  object 
 2   Region               22754 non-null  object 
 3   Subregion            22754 non-null  object 
 4   Indicator            22754 non-null  object 
 5   Dimension            22754 non-null  object 
 6   Category             22754 non-null  object 
 7   Sex                  22754 non-null  object 
 8   Age                  22754 non-null  object 
 9   Year                 22754 non-null  int64  
 10  Unit of measurement  22754 non-null  object 
 11  VALUE                22754 non-null  float64
 12  Source               22754 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 16.5 MB


In [63]:
theftCrime.head()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
0,ARM,Armenia,Asia,Western Asia,Offences,by type of offence,Corruption,Total,Total,2013,Counts,782.0,CTS
1,AUT,Austria,Europe,Western Europe,Offences,by type of offence,Corruption,Total,Total,2013,Counts,3439.0,CTS
2,CHE,Switzerland,Europe,Western Europe,Offences,by type of offence,Corruption,Total,Total,2013,Counts,4884.0,CTS
3,CHL,Chile,Americas,Latin America and the Caribbean,Offences,by type of offence,Corruption,Total,Total,2013,Counts,339.0,CTS
4,COL,Colombia,Americas,Latin America and the Caribbean,Offences,by type of offence,Corruption,Total,Total,2013,Counts,23483.0,CTS


In [64]:
theftCrime.shape

(22754, 13)

In [65]:
theftCrime.describe()

,Year,VALUE
count,22754.000000,2.275400e+04
mean,2015.348159,2.675275e+04
std,5.059154,2.134465e+05
min,2003.000000,0.000000e+00
25%,2013.000000,6.000000e+00
50%,2017.000000,1.272963e+02
75%,2019.000000,1.484817e+03
max,2022.000000,7.026802e+06


### Preprocessing dei dati
Filtrimo il dataframe utilizzando come unità solo 'Counts' inevce che 'Rate per 100,000 population'. Effettuiamo quindi una prima scrematura dei dati

In [66]:
theftCrime = theftCrime[theftCrime['Unit of measurement'] != "Rate per 100,000 population"]

In [67]:
theftCrime.shape

(11377, 13)

Analizzando il dataframe, le feature 'Age', 'Sex', 'Iso3_code', 'Dimension', 'Unit of measurement'e 'Source' non sono rilevanti per il nostro modello, perciò effetuiamo la loro rimozione.

Rinominiamo inoltre la colonna `VALUE` in `Value`

In [68]:
theftCrime.rename(columns={'VALUE':'Value'}, inplace=True)
theftCrime = theftCrime.drop(['Unit of measurement', 'Sex', 'Iso3_code', 'Age', 'Source', 'Dimension', 'Indicator'], axis=1)

In [69]:
theftCrime.head()

,Country,Region,Subregion,Category,Year,Value
0,Armenia,Asia,Western Asia,Corruption,2013,782.0
1,Austria,Europe,Western Europe,Corruption,2013,3439.0
2,Switzerland,Europe,Western Europe,Corruption,2013,4884.0
3,Chile,Americas,Latin America and the Caribbean,Corruption,2013,339.0
4,Colombia,Americas,Latin America and the Caribbean,Corruption,2013,23483.0


Notimao inoltre che, dalle informazioni ricevute nel paragrafo precedente, la maggior parte delle feature tranne `Year` e `Value` sono di tipo `Object`, e questo perchè paython converte automaticamente in tipo objet tuttin i dati che non sono interi. Dunque cambiamo i vari tipi delle feature.
Inoltre, rimuoviamo, se vi sono, le varie righe conteneti valori nulli.

In [70]:
theftCrime["Country"] = theftCrime["Country"].astype(pd.StringDtype())
theftCrime["Region"] = theftCrime["Region"].astype(pd.StringDtype())
theftCrime["Subregion"] = theftCrime["Subregion"].astype(pd.StringDtype())
theftCrime["Category"] = theftCrime["Category"].astype(pd.StringDtype())
theftCrime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11377 entries, 0 to 11376
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    11377 non-null  string 
 1   Region     11377 non-null  string 
 2   Subregion  11377 non-null  string 
 3   Category   11377 non-null  string 
 4   Year       11377 non-null  int64  
 5   Value      11377 non-null  float64
dtypes: float64(1), int64(1), string(4)
memory usage: 622.2 KB


In [71]:
theftCrime.dropna()
theftCrime.shape

(11377, 6)

Notiamo che il nostro dataset non contiene valori nulli in quanto la shape non è cambiata.

Contiamo anche il numero di paesi nel nostro dataset.

In [72]:
print(f"Numero totale di paesi: {theftCrime['Country'].unique().size}")

Numero totale di paesi: 157


## Crimini per omicidio

In [3]:
import os.path
file = "data_cts_intentional_homicide.csv"
if not os.path.exists(file):
    print("Missing dataset")
else:
  homicideCrime = pd.read_csv(file, index_col=False, encoding='latin1')

In [74]:
homicideCrime.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117070 entries, 0 to 117069
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Iso3_code            117070 non-null  object 
 1   Country              117070 non-null  object 
 2   Region               117070 non-null  object 
 3   Subregion            117070 non-null  object 
 4   Indicator            117070 non-null  object 
 5   Dimension            117070 non-null  object 
 6   Category             117070 non-null  object 
 7   Sex                  117070 non-null  object 
 8   Age                  117070 non-null  object 
 9   Year                 117070 non-null  int64  
 10  Unit of measurement  117070 non-null  object 
 11  VALUE                117070 non-null  float64
 12  Source               117070 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 87.4 MB


In [75]:
homicideCrime.head()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
0,ARM,Armenia,Asia,Western Asia,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,35.0,CTS
1,CHE,Switzerland,Europe,Western Europe,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,28.0,CTS
2,COL,Colombia,Americas,Latin America and the Caribbean,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,15053.0,CTS
3,CZE,Czechia,Europe,Eastern Europe,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,69.0,CTS
4,DEU,Germany,Europe,Western Europe,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,455.0,CTS


In [76]:
homicideCrime.shape

(117070, 13)

In [77]:
homicideCrime.describe()

,Year,VALUE
count,117070.000000,117070.000000
mean,2014.886162,733.792988
std,6.167995,9906.653113
min,1990.000000,0.000000
25%,2012.000000,0.243000
50%,2016.000000,2.556600
75%,2019.000000,20.935593
max,2023.000000,457945.484991


### Preprocessing dei dati
Filtrimo il dataframe utilizzando come unità solo 'Counts' inevce che 'Rate per 100,000 population'. Effettuiamo quindi una prima scrematura dei dati

In [78]:
homicideCrime = homicideCrime[homicideCrime['Unit of measurement'] != "Rate per 100,000 population"]
homicideCrime.shape

(62850, 13)

Così facendo rimuovimao, come precedentemente, quasi la metà dei dati e allegeriamo così il dataset.

Come prima, rimuoviamo anche da questo dataset le feature non rilevanti per lo studio, e rinominiamo la feature `VALUE` come prima in `Value`.

Inoltre, effettuiamo anche la conversione delle feature da `Object` in `String`.

In [79]:
homicideCrime.rename(columns={'VALUE':'Value'}, inplace=True)
homicideCrime = homicideCrime.drop(['Unit of measurement', 'Sex', 'Iso3_code', 'Age', 'Source', 'Dimension', 'Indicator'], axis=1)
homicideCrime["Country"] = homicideCrime["Country"].astype(pd.StringDtype())
homicideCrime["Region"] = homicideCrime["Region"].astype(pd.StringDtype())
homicideCrime["Subregion"] = homicideCrime["Subregion"].astype(pd.StringDtype())
homicideCrime["Category"] = homicideCrime["Category"].astype(pd.StringDtype())
homicideCrime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62850 entries, 0 to 62849
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    62850 non-null  string 
 1   Region     62850 non-null  string 
 2   Subregion  62850 non-null  string 
 3   Category   62850 non-null  string 
 4   Year       62850 non-null  int64  
 5   Value      62850 non-null  float64
dtypes: float64(1), int64(1), string(4)
memory usage: 3.4 MB


In [80]:
homicideCrime.head()

,Country,Region,Subregion,Category,Year,Value
0,Armenia,Asia,Western Asia,National citizens,2013,35.0
1,Switzerland,Europe,Western Europe,National citizens,2013,28.0
2,Colombia,Americas,Latin America and the Caribbean,National citizens,2013,15053.0
3,Czechia,Europe,Eastern Europe,National citizens,2013,69.0
4,Germany,Europe,Western Europe,National citizens,2013,455.0


In [81]:
homicideCrime.dropna()
homicideCrime.shape

(62850, 6)

Notimao anche questa volta che la shape del dataset non è cambiata dopo aver effettuato la `dropna()`, il che vuol dire non vi sono presenti righr con valori nulli.



Controlliamo il numero di paesi del dataset:

In [85]:
print(f"Numero totale di paesi: {homicideCrime['Country'].unique().size}")
countries_theft = set(theftCrime['Country'])
countries_homicide = set(homicideCrime['Country'])

# Confronta gli insiemi
if countries_theft == countries_homicide:
    print("Entrambi i DataFrame contengono gli stessi paesi.")
else:
    print("I DataFrame non contengono gli stessi paesi.")

Numero totale di paesi: 231
I DataFrame non contengono gli stessi paesi.


In questo caso, il numero di peasi è lo stesso del dataset precedente, ma non contiene gli stessi paesi.

Dunque, per poter procedere verso il merging dei vari dataset per creare uno solo, bisogna rimuovere da `theftCrime` tutte le righe rigurdanti i paesi non contenuti in `homicideCrime`.

Controlliamo intanto i paesi che differenziano da un dataset all'altro.

In [86]:
different_in_theft = countries_theft - countries_homicide
different_in_homicide = countries_homicide - countries_theft

if different_in_theft:
    print(f"Paesi in theftCrime ma non in homicideCrime: {different_in_theft}")
    print(f"Numero: {len(different_in_theft)}")
if different_in_homicide:
    print(f"Paesi in homicideCrime ma non in theftCrime: {different_in_homicide}")
    print(f"Numero: {len(different_in_homicide)}")

Paesi in theftCrime ma non in homicideCrime: {'Guinea', 'Benin', 'Sudan', 'CÃ´te dâ\x80\x99Ivoire', 'Senegal'}
Numero: 5
Paesi in homicideCrime ma non in theftCrime: {'Afghanistan', 'Eritrea', 'World', 'Montserrat', 'San Marino', 'Micronesia', 'South Africa', 'Turks and Caicos Islands', 'Anguilla', 'Ethiopia', 'Haiti', 'Martinique', 'Aruba', 'Northern America', 'Sub-Saharan Africa', 'Guam', 'Angola', 'All Oceania', 'Palau', 'Cuba', 'Northern Europe', 'Tunisia', 'Niger', 'Malawi', 'Cayman Islands', 'Isle of Man', 'Central Asia', 'Kiribati', 'Guadeloupe', 'Micronesia (Federated States of)', 'All Europe', 'Faroe Islands', 'Northern Africa', 'Iraq', 'All Africa', 'New Caledonia', 'All Asia', 'RÃ©union', 'Cambodia', 'All Americas', 'American Samoa', 'Iraq (Kurdistan Region)', 'Southern Asia', 'Zambia', 'Cook Islands', 'Vanuatu', 'Gibraltar', 'United States Virgin Islands', 'Eastern Asia', 'Mayotte', 'Greenland', 'Western Asia', 'French Guiana', 'Polynesia', 'Southern Europe', 'Saint Pierre 

In [88]:
common_countries = countries_theft & countries_homicide
print(f"Numero Paesi in comune: {len(common_countries)}")

Numero Paesi in comune: 152


Il numero di paesi in comune è 152, quindi muteremo i due dataset filtrandoli in base ai paesi in comune tra i due.

In [91]:
theftCrime = theftCrime[theftCrime['Country'].isin(common_countries)]
homicideCrime = homicideCrime[homicideCrime['Country'].isin(common_countries)]
print(f"theftCrime: {theftCrime.shape}")
print(f"homicideCrime: {homicideCrime.shape}")

theftCrime: (11355, 6)
homicideCrime: (60234, 6)
